In [7]:
from ai.TrashDetector import TrashDetector
from ai.DataLoader import DetectorDataLoader
from env import *

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
ETA = 1e-3
EPOCHS = 50

In [8]:
dloader = DetectorDataLoader(DETECTOR_TRAIN_DATA_PATH, DETECTOR_CAT, noise=True)

Number of data batch: 283
Number of batch: 18


In [3]:
detector = TrashDetector().cuda()

In [5]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(detector.parameters(), lr=ETA)

In [ ]:
for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    
    detector.train()
    
    for x_batch, y_batch in dloader.next_batch():
        x = torch.FloatTensor(x_batch).cuda()
        y = torch.LongTensor(y_batch).cuda()
        
        logps = detector(x)
        loss = criterion(logps, y)
        train_loss += loss.item()
        
        with torch.no_grad():
            ps = torch.exp(logps)
            ps_k, top_k = ps.topk(1, dim=1)
            train_acc += torch.mean((top_k == y.view(*top_k.size())).type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    detector.eval()
    
    train_loss /= len(dloader)
    train_acc /= len(dloader)
    
    print(f"Epochs {e+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.8f}")
    print(f"Train acc: {train_acc:.4f}")
    
detector.save(DET_CKPT_PATH)
    